# Avalanche - A Python Library for Continual Learning

References:


[Github repository](https://github.com/ContinualAI/avalanche)

[Official website](https://avalanche.continualai.org/)



In [1]:
# Install Avalanche 
!pip install git+https://github.com/ContinualAI/avalanche.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 2.6 MB/s 
     |████████████████████████████████| 121 kB 54.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532 kB 35.7 MB/s 
     |████████████████████████████████| 139 kB 47.1 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 180 kB 54.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [2]:
import torch
# use CrossEntropyLoss
from torch.nn import CrossEntropyLoss
# use stochastic GD optimizer
from torch.optim import SGD
# import the PermutedMNIST dataset
from avalanche.benchmarks.classic import PermutedMNIST
# import the SimpleMLP dataset
from avalanche.models import SimpleMLP
# we will use Naive training strategy
from avalanche.training.strategies import Naive

In [3]:
# Configure the device settings
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model instantiation
model = SimpleMLP(num_classes = 10)

In [4]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

In [6]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

In [7]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [00:42<00:00, 44.10it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4912
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8605
100%|██████████| 1875/1875 [00:42<00:00, 44.17it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2590
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9247
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:06<00:00, 47.22it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1835
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9474
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 313/313 [00:06<00:00, 47.53it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 2.5601
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.1092
-- S

## Results

In [8]:
results

[{'Loss_Epoch/train_phase/train_stream/Task000': 0.2590019486248493,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.18345425557643175,
  'Loss_Exp/eval_phase/test_stream/Task001/Exp001': 2.560053448486328,
  'Loss_Exp/eval_phase/test_stream/Task002/Exp002': 2.499099814987183,
  'Loss_Stream/eval_phase/test_stream/Task000': 0.18345425557643175,
  'Loss_Stream/eval_phase/test_stream/Task001': 2.560053448486328,
  'Loss_Stream/eval_phase/test_stream/Task002': 2.499099814987183,
  'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9246833333333333,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9474,
  'Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001': 0.1092,
  'Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002': 0.1514,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.9474,
  'Top1_Acc_Stream/eval_phase/test_stream/Task001': 0.1092,
  'Top1_Acc_Stream/eval_phase/test_stream/Task002': 0.1514},
 {'Loss_Epoch/train_phase/train_stream/Task000': 0.2590019486248493